In [47]:
from openai import OpenAI
import json
import os

GPT_MODEL_4 = "gpt-4-0125-preview"
OPEN_API_KEY = os.getenv("OPENAI_API_KEY")

client = OpenAI()
model = GPT_MODEL_4

def ask(prompt, client, model, temperature = 0):
    response = None
    
    response = client.chat.completions.create(
      model=model,
      messages=prompt,
      temperature=temperature,
    )

    return response.choices[0].message.content

def askJSON(prompt, client, model, temperature = 0):
    response = None
    
    response = client.chat.completions.create(
      model=model,
      messages=prompt,
      temperature=temperature,
      response_format={ "type": "json_object" },
    )

    return response.choices[0].message.content

In [48]:
def write_string_to_file(filename, content):
    try:
        with open(filename, 'w') as file:
            file.write(content)
        print(f"String has been written to {filename}")
    except IOError as e:
        print(f"An error occurred while writing to the file: {e}")

In [49]:
import chardet

def read_file_content(file_path):
    # Detect the encoding
    with open(file_path, 'rb') as file:
        raw_data = file.read()
        result = chardet.detect(raw_data)
        encoding = result['encoding']
    
    # Read the file with the detected encoding
    try:
        with open(file_path, 'r', encoding=encoding) as file:
            content = file.read()
        return content
    except FileNotFoundError:
        return f"Error: The file at path {file_path} was not found."
    except UnicodeDecodeError:
        return f"Error: The file at path {file_path} cannot be decoded with the {encoding} encoding."
    except IOError:
        return f"Error: An I/O error occurred while reading the file at path {file_path}."


In [50]:
use_case_list = [
    "Add new topic.txt",
"AddLesson.txt",
"AddQuestion .txt",
"EditTest.txt",
"Handbook.txt",
"Learning days history.txt",
"ListenToPronunciation.txt",
"Review test.txt",
"WordSortingTest.txt",
"Flashcard.txt",
"Login.txt",
"Registry.txt"
]

In [51]:
BASELINE_GENERATE_TC = """
Generate test cases based on given use case specifications and test scenarios.
"""

In [52]:
use_case_directory_path = r"D:\GPT-testing\SpecificationData\MatchaEnglishWebsite"
test_scenario_directory_path = r"D:\Dissertation-GPT\evaluate\baseline\ts\matcha"
save_testcase_path = r"D:\Dissertation-GPT\evaluate\baseline\tc"

In [53]:

for usecase in use_case_list:
  base_name = os.path.basename(usecase)
  usecase_name = os.path.splitext(base_name)[0]
  usecase_path = os.path.join (use_case_directory_path,f"{usecase_name}.txt" )
  testscenario_path = os.path.join (test_scenario_directory_path,f"{usecase_name}.txt" )
  testscenario = read_file_content(testscenario_path)
  content = "Use case:" + read_file_content(usecase_path) + "\nTest scenarios:\n"+ testscenario
  promptExtractCondition = [
    { "role": "system", "content": BASELINE_GENERATE_TC},
    { "role": "user", "content": content}
  ]
  gpt_response = ask(promptExtractCondition, client, model)
  print(content)
  print(gpt_response)
  print()
  print()
  print()

  testcase_path = os.path.join (save_testcase_path,f"{usecase_name}.txt" )
  write_string_to_file(testcase_path, gpt_response)

Use case:use case id: U17
use case name: adding new topic
objective: This use case allows administrator to add a new english learning topic.
actor: administrator

basic flow:
step 1: Administrator selects "Add new topic" tab on the navigation bar.
step 2: The system redirects Administrator to the adding topic viewing page.
step 3: Administrator adds the name of the topic in the name field.
step 4: Administrator adds the description of the topic in the description field.
step 5: Administrator selects "Save" to create a new topic.

Alternative flow: Administrator adds an already used name for the topic.
At step 3 of the basic flow: Administrator adds a name that is already used in the existed topics in the name field.
step 4: Administrator adds the description of the topic in the description field.
step 5: Administrator selects "Save" to create a new topic.
step 6: The system shows notification that the name is already used, prompting Administrator to add another name in the name field.
